In [1]:
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import os

class FFTTransform:
    def __call__(self, img):
        img_tensor = transforms.functional.to_tensor(img)
        padded_img = F.pad(img_tensor, (0, 6, 0, 6))
        fft_result = torch.fft.fft2(padded_img)
        fft_magnitude = torch.abs(fft_result)
        fft_magnitude = fft_magnitude / fft_magnitude.max()
        return fft_magnitude

transform = transforms.Compose([
    FFTTransform(),
])

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_images = trainset.data.unsqueeze(1)
test_images = testset.data.unsqueeze(1)

torch.save({
    'train_images': train_images,
    'train_labels': trainset.targets,
    'test_images': test_images,
    'test_labels': testset.targets,
}, './dataset/transformed_mnist.pt')

print("Dataset saved to disk.")


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:02<00:00, 3.36MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 194kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:02<00:00, 759kB/s] 


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 13.6MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Dataset saved to disk.


CIFR10

In [2]:
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import os

class FFTTransform:
    def __call__(self, img):
        img_tensor = transforms.functional.to_tensor(img)
        padded_img = F.pad(img_tensor, (0, 6, 0, 6))
        fft_result = torch.fft.fft2(padded_img)
        fft_magnitude = torch.abs(fft_result)
        fft_magnitude = fft_magnitude / fft_magnitude.max()
        return fft_magnitude

transform = transforms.Compose([
    FFTTransform(),
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_images = torch.stack([img for img, _ in trainset])
test_images = torch.stack([img for img, _ in testset])

torch.save({
    'train_images': train_images,
    'train_labels': torch.tensor(trainset.targets),
    'test_images': test_images,
    'test_labels': torch.tensor(testset.targets),
}, './dataset/transformed_cifar10_RGB.pt')

print("Dataset saved to disk.")

100%|██████████| 170M/170M [00:36<00:00, 4.64MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Dataset saved to disk.


In [3]:
import torch
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F

import os

class HSVTransform:
    def __call__(self, img):
        img = img.convert("HSV")  # Converte l'immagine in HSV
        img_tensor = TF.to_tensor(img)  # Converte in tensore PyTorch
        padded_img = F.pad(img_tensor, (0, 6, 0, 6))
        fft_result = torch.fft.fft2(padded_img)
        fft_magnitude = torch.abs(fft_result)
        fft_magnitude = fft_magnitude / fft_magnitude.max()
        return fft_magnitude

# Definiamo la trasformazione
transform = transforms.Compose([
    HSVTransform(),
])

# Carichiamo il dataset con la nuova trasformazione
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Convertiamo le immagini in tensori
train_images = torch.stack([img for img, _ in trainset])
test_images = torch.stack([img for img, _ in testset])

# Salviamo il dataset trasformato
torch.save({
    'train_images': train_images,
    'train_labels': torch.tensor(trainset.targets),
    'test_images': test_images,
    'test_labels': torch.tensor(testset.targets),
}, './dataset/transformed_cifar10_HSV.pt')

print("Dataset in formato HSV salvato su disco.")


Files already downloaded and verified
Files already downloaded and verified
Dataset in formato HSV salvato su disco.


In [4]:
import torch
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F

import os

class HSVTransform:
    def __call__(self, img):
        img = img.convert("HSV")  # Converte l'immagine in HSV
        img_tensor = TF.to_tensor(img)  # Converte in tensore PyTorch
        return img_tensor

# Definiamo la trasformazione
transform = transforms.Compose([
    HSVTransform(),
])

# Carichiamo il dataset con la nuova trasformazione
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Convertiamo le immagini in tensori
train_images = torch.stack([img for img, _ in trainset])
test_images = torch.stack([img for img, _ in testset])

# Salviamo il dataset trasformato
torch.save({
    'train_images': train_images,
    'train_labels': torch.tensor(trainset.targets),
    'test_images': test_images,
    'test_labels': torch.tensor(testset.targets),
}, './dataset/cifar10_HSV.pt')

print("Dataset in formato HSV salvato su disco.")

Files already downloaded and verified
Files already downloaded and verified
Dataset in formato HSV salvato su disco.


In [1]:
import torch
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F

import os


# Definiamo la trasformazione
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Carichiamo il dataset con la nuova trasformazione
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Convertiamo le immagini in tensori
train_images = torch.stack([img for img, _ in trainset])
test_images = torch.stack([img for img, _ in testset])

# Salviamo il dataset trasformato
torch.save({
    'train_images': train_images,
    'train_labels': torch.tensor(trainset.targets),
    'test_images': test_images,
    'test_labels': torch.tensor(testset.targets),
}, './dataset/cifar10_RGB.pt')

print("Dataset in formato HSV salvato su disco.")

Files already downloaded and verified
Files already downloaded and verified
Dataset in formato HSV salvato su disco.
